In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from tqdm import notebook
import datetime
import time
import pandas as pd

In [18]:
#드라이버 불러온 후 전체화면 크기로 설정
driver = webdriver.Chrome('./chromedriver.exe')
driver.maximize_window()

### CSV update 코드  
- ```file_name```과 ```header(column)```으로 들어갈 값을 작성

In [19]:
def update_csv(f_name, data_list):
    df = pd.DataFrame(data_list)
    df.to_csv(f_name, index=False, mode='a')


### jumpit site 크롤링 코드  
- Category 1~21까지 반복하며 나오는 공고들을 하나씩 클릭
- 데이터 저장 형태는 2중 딕셔너리 리스트 형태로 저장, 사이트마다 다를 수 있기 때문에 조정필요  
-> ```{'회사이름_모집분야': {'기술스택':['java', 'python'],'주요업무':[], ...}}```   
-> CSV 형태로 저장 **[col: 회사이름,기술스택, 주요업무, ...]**
- 인터넷 성능에 따라 ```implicityly_wait()```의 시간을 조정하면 될 것 같습니다.

In [20]:
def jumpit_crawling(start_category, start_xpath_location):

    crawling_data = []
    max_category = 21
    URL = 'https://www.jumpit.co.kr/positions'
    for jobCategory in range(start_category, max_category+1):

        driver.get(URL + f'?jobCategory={jobCategory}')
        driver.implicitly_wait(time_to_wait=2)
        
        xpath_location = start_xpath_location

        #페이지의 공고들을 반복하여 클릭해서 들어감
        for i in notebook.tqdm(range(1000), desc = '1000개 크롤링'):
            xpath_location += 1
            driver.implicitly_wait(time_to_wait=3)

            #회사이름, 모집 분야
            company_name = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[1]/section/div[{xpath_location}]/a/div[3]/div').text
            field_recruitment = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[1]/section/div[{xpath_location}]/a/div[3]/h2').text


            #공고 클릭
            driver.find_element(By.XPATH, f'//*[@id="root"]/main/div/div[1]/section/div[{xpath_location}]').click()
            driver.implicitly_wait(time_to_wait=2)


            #회사이름, 기술스택, 주요업무, 자격요건, 우대사항, 복지 및 혜택, 경력, 학력, 마감일, 근무지역 dict_data 저장
            topic_dic = {'회사이름':company_name+field_recruitment}
            for i in range(1, 6):
                topic = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[{i}]/dt').text
                element = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[{i}]/dd/pre').text

                topic_dic[topic] = element
            
            data = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[3]').text.split('\n')[1:-1]
            for i in range(0, len(data), 2):
                topic_dic[data[i]] = data[i+1]

            crawling_data.append(topic_dic)


            #20개에 한 번씩 저장
            if len(crawling_data) == 20:
                update_csv('크롤링_점핏.csv', crawling_data)  
                print('20 topic -> CSV file update')
                crawling_data = []
                
            driver.back()

### 데이터 수집 및 시간 측정

In [22]:
#수행해야할 category_number와 xpath_location 입력

start = time.time()
jumpit_crawling(1, 0)
end = time.time()

result = datetime.timedelta(seconds=end-start)
print(result)

20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update
20 topic -> CSV file update



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="root"]/main/div/div[1]/section/div[674]/a/div[3]/div"}
  (Session info: chrome=114.0.5735.134)
